## Set up Titan model on Bedrock

In [2]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --quiet --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.0 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.0 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.3.0 which is incompatible.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
pyasn1-modules 0.2.8 requires pyasn1<0.5.0,>=0.4.6, but you have pyasn1 0.5.0 which is incompatib

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-west-2.amazonaws.com)


In [4]:
boto3_bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '80af66a8-0109-4102-b4bb-4d1e01671104',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 28 Aug 2023 17:32:46 GMT',
   'content-type': 'application/json',
   'content-length': '1166',
   'connection': 'keep-alive',
   'x-amzn-requestid': '80af66a8-0109-4102-b4bb-4d1e01671104'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-i

## Install and import packages

In [5]:
%pip install --upgrade --quiet sagemaker
%pip install --upgrade --quiet langchain
%pip install --upgrade --quiet openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip uninstall --quiet typing_extensions --yes
%pip install --upgrade --quiet typing_extensions

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.14.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import openai
import pandas as pd
import time
# from dotenv import load_dotenv
from langchain.chat_models import ChatAnyscale, ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os
# from multichoice import Multichoice
from prompt_mgr import PromptMgr

## Read input dataset and create prompt

In [8]:
df = pd.read_json('resources/datasets/val_sentence_pairs.json')
df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2


In [9]:
pm = PromptMgr(src_dir = 'resources/environments/answer_first/prompts')

In [10]:
def query_model(row, prompt_mgr, swap_answers = False, bedrock_model_id = 'amazon.titan-tg1-large'):
    if swap_answers: 
        prompt = prompt_mgr.bind('consistent').render(
                    article_sent=row['article_sent'], 
                    option_a=row['incorrect_sent'],
                    option_b=row['correct_sent'])
    else: 
        prompt = prompt_mgr.bind('consistent').render(
                    article_sent=row['article_sent'], 
                    option_a=row['correct_sent'],
                    option_b=row['incorrect_sent'])
    system_prompt = prompt_mgr.bind('system').render()
    prompt_data = f"{system_prompt}\n{prompt}"
    
    
    body = json.dumps({
        "inputText": prompt_data, 
        "textGenerationConfig":{
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
            }
    }) 
    modelId = bedrock_model_id
    accept = 'application/json'
    contentType = 'application/json'
    
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    outputText = response_body.get('results')[0].get('outputText')
    
    return {'output': outputText} 

## Call Titan model

In [11]:
# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], pm, swap_answers=True)
    results_list.append(result)

# Create a new DataFrame from the results list
results_df = pd.DataFrame(results_list)

In [12]:
results_df.loc[0]['output']

'B'

In [13]:
results_df.to_json('Titan-model-output-swapped.json')

In [14]:
# Create an empty list to store the results
results_list_no_swap = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], pm, swap_answers=False)
    results_list_no_swap.append(result)

# Create a new DataFrame from the results list
results_df_no_swap = pd.DataFrame(results_list_no_swap)

In [15]:
results_df_no_swap.loc[0]['output']

'B'

In [16]:
results_df_no_swap.to_json('Titan-model-output-no-swap.json')

## Postprocessing and analysis

In [19]:
titan_swapped_df  = pd.read_json('Titan-model-output-swapped.json')
titan_no_swap_df  = pd.read_json('Titan-model-output-no-swap.json')

In [20]:
titan_swapped_df.head()

,output
0,B
1,B
10,B
100,B
101,B


In [21]:
titan_swapped_df.shape

(373, 1)

In [22]:
titan_swapped_df.rename(columns={'output': 'titan_swapped_output'}, inplace=True)
titan_no_swap_df.rename(columns={'output': 'titan_no_swap_output'}, inplace=True)

In [24]:
input_df = df

combined_df = pd.concat([input_df, 
                         titan_swapped_df, 
                         titan_no_swap_df,], axis=1)
combined_df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id,titan_swapped_output,titan_no_swap_output
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0,B,B
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1,B,B
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1,B,A
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2,B,A
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2,B,A


In [25]:
def is_correct(row): 
    if (row[0] == 'B' and row[1] == 'A'):
        return 'Y'
    if (row[0] == row[1]):
        return row[0]*2
    return 'N'

In [26]:
combined_df['titan_analysis_result'] = combined_df[['titan_swapped_output', 'titan_no_swap_output']].apply(is_correct, axis=1)

In [27]:
def acc_bias(col):
    results = combined_df[col].value_counts()/len(combined_df)
    acc = results['Y']
    if 'AA' not in results:
        results['AA'] = 0
    if 'BB' not in results: 
        results['BB'] = 0
        
    bias = abs(results['AA'] - results['BB'])
    aa_ratio = results['AA'] * 100
    bb_ratio = results['BB'] * 100
    if results['AA'] > results['BB']:
        towards = 'A'
    else:
        towards = 'B'
        
    return acc, aa_ratio, bb_ratio, bias, towards

In [28]:
for m in ['titan']:
    acc, aa_ratio, bb_ratio, bias, towards = acc_bias(m+'_analysis_result')
    acc = acc*100
    bias = bias*100
    print(f'{m}:   \tAccuracy: {acc:.1f}%  \tAA: {aa_ratio:.1f}%\tBB: {bb_ratio:.1f}%\tBias: {bias:.1f}% towards {towards}')

titan:   	Accuracy: 65.4%  	AA: 2.1%	BB: 28.2%	Bias: 26.0% towards B
